<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

#  Классификация комментариев на позитивные и негативные

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

In [1]:
# Загрузим необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from pymystem3 import Mystem
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# Загрузим и изучим данные
data = pd.read_csv('/datasets/toxic_comments.csv')
display(data.shape)
display(data.head())
data.info()

(159571, 2)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [3]:
# Изучим баланс классов
balance = data['toxic'].sum() / len(data)
balance

0.10167887648758234

Мы видим, что классы несбалансированы: токсичных комментариев примерно 10%

In [4]:
# Напишем функции лемматизации текста и отчистки от всех символов, кроме английских букв
#m = Mystem()
corpus = data['text']

def lemmatize(text):
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
    return lemm_text


def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())

data['lemm_text'] = corpus.apply(clear_text)
#data['lemm_text'] = corpus.apply(lemmatize)

Проведем разбиение на обучающую и тестовую выборки

In [5]:
data_train, data_test = train_test_split(data, test_size=0.25, random_state=12345)

In [6]:
# Вычислим TF-IDF для обучающего и тестового корпусов текстов
train_corpus = data_train['lemm_text'].values.astype('U')
train_labels = data_train['toxic']
 
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
 
tfidf_train = count_tf_idf.fit_transform(train_corpus)


test_corpus = data_test['lemm_text'].values.astype('U')
test_labels = data_test['toxic']

tfidf_test = count_tf_idf.transform(test_corpus)

## Обучение

Обучим 3 различные модели, сформируем прогнозы, рассчитаем метрику f1 для тестовой выборки

In [7]:
model_1 = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model_1.fit(tfidf_train, train_labels)
predictions = model_1.predict(tfidf_test)
 
print(f1_score(test_labels, predictions))

0.7588081774684645


In [8]:
model_2 = DecisionTreeClassifier()
model_2.fit(tfidf_train, train_labels)
predictions = model_2.predict(tfidf_test)
 
print(f1_score(test_labels, predictions))

0.7117315699240754


In [9]:
model_3 = RandomForestClassifier()
model_3.fit(tfidf_train, train_labels)
predictions = model_3.predict(tfidf_test)
 
print(f1_score(test_labels, predictions))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6798780487804879


## Выводы

Мы обучили три различные модели LogisticRegression, DecisionTreeClassifier, RandomForestClassifier. Для каждой модели сформировали прогнозы, а также рассчитали метрику f1. Наибольшее значение метрики f1 получилось у модели LogisticRegression, наименьшее - у модели RandomForestClassifier.

Используя написанный код мы можем классифицировать комментарии на позитивные и негативные, не тратя время на их прочтение, что позволит сохранить рабочее время работников интернет-магазина при оценке комментариев.